## mlmsynth Example Jupyter Notebook
This notebook shows how to
* Properly format input data in json files for applying PU learning.
* Run the mlmsynth implementation of PU learning and predict "synthesizability scores" for not-yet-synthesized materials.
* Compare synthesizability scores between "parent" bulk crystals and "child" 2D phases to identify new bulk crystals that can be synthesized and exfoliated into a new 2D material.
* Use unsupervised clustering algorithms to analyze results.

In [ ]:
import pandas as pd
import numpy as np

### Take a look at the format of input data.
Each row corresponds to a material and each column is a feature from either elemental data or density functional theory calculations.
There **must** be a column called "PU_label" that labels each material as positive/synthesized (1) or not-yet-synthesized/unlabeled (0).

In [ ]:
df = pd.read_json('test_files/MAX_dataset.json')  # Data must be in json format

df2 = pd.read_json('test_files/MX_dataset.json')
df.head()

### Do k-fold cross validation with bagged decision tree base classifiers.
Run PU learning on a sample data set. n_repeats and n_bags should be large values for production runs. Here we use small values so the example runs quickly.

In [ ]:
from mlmsynth.learners import PULearner

In [ ]:
pul = PULearner()
n_splits = 3  # 3-fold CV
n_repeats = 5  # Repeat the entire kfold CV 10 times for averaging
n_bags = 5  # 10 bags for bootstrap aggregating.

pu_stats_max = pul.cv_baggingDT('test_files/MAX_dataset.json', splits=n_splits, repeats=n_repeats, bags=n_bags)

### Get the synthesizability predictions from PU learning.
"Synthesizability scores" (a number between 0 and 1) indicate the aggregated PU learning model prediction of how "synthesizable" a material is. We say that a material is "likely synthesizable" if the synth_score > 0.5 and "not likely synthesizable" if the synth_score < 0.5.

In [ ]:
df1 = pul.df_U.copy()  # get a copy of the dataframe of nlabeled samples
df1['synth_score'] = pu_stats_max['prob']  # pu_stats['prob'] stores the synthesizability score of the unlabeled samples
df1.head()

### Compare PU learning synthesizability predictions for "parent" bulk materials and "child" 2D materials.
After running the PU analysis on data sets of "parent" and "child" materials, we use clustering algorithms like k-means to identify clusters of synthesizable (parent, child) pairs. For example, we might want to know if there is an unlabeled parent bulk crystal that is highly synthesizable **and** it can be exfoliated into a not-yet-synthesized child 2D phase with an independently high synthesizability.

In [ ]:
# Run PU learning on the "child" data
pu_stats_mx = pul.cv_baggingDT('test_files/MX_dataset.json', splits=n_splits, repeats=n_repeats, bags=n_bags)
df2 = pul.df_U.copy()
df2['synth_score'] = pu_stats_mx['prob']

In [ ]:
from mlmsynth.learners import PUInteract

In [ ]:
merge = ('M', 'X', 'n')  # Specify a column or tuple of columns that link (parent, child) pairs
feats = ('a', 'E_form', 'E_coh', 'synth_score')  # a tuple of column names of features to consider in clustering

# We pass the original data and the results from PU learning
pui = PUInteract('test_files/MAX_dataset.json', pu_stats_max, 'test_files/MX_dataset.json', pu_stats_mx, merge_on=merge, feats=feats)

In [ ]:
df = pui.merged_df
kmeans_output = pui.do_kmeans(n_clusters=2)  # Run k-means clustering with 2 clusters
df['kmeans_labels'] = kmeans_output['cluster_labels']
df['kmeans_labels'].value_counts()

### Clustering
k-means, Gaussian mixture models, and Bayesian Gaussian mixture models can be used to predict labels for the (parent, child) pairs. With some fine-tuning, these models can cluster the samples into "high synthesizability" and "low synthesizablity" clusters. Analyzing the distributions of features within the two clusters may be illuminating.

In [ ]:
gmm_output = pui.do_gmixture(n_components=2)  # Gaussian mixture model
df['gmm_labels'] = gmm_output['gmm_labels']
bgm_output = pui.do_bgm(n_components=6)  # Bayesian gaussian mixture
df['bgm_labels'] = bgm_output['bgm_labels']
print(df['gmm_labels'].value_counts())
print(df['bgm_labels'].value_counts())